# Camada Gold

## 1.0 Agregações e métricas

### 1.1 Visualizando arquivo na Camada Silver

In [0]:
%fs
ls dbfs:/mnt/Cotacao_Dolar/Silver

### 1.2 Carregando o arquivo parquet em DataFrame

In [0]:
cotacao_dolar_df = (
    spark.read.format("delta")
    .load("dbfs:/mnt/Cotacao_Dolar/Silver/Ft_Cotacao_Dolar/")
)

### 1.3 Cálculo da média mensal das cotações de compra e venda do dólar, agrupando por ano e mês

In [0]:
from pyspark.sql import functions as F

cotacao_dolar_df = cotacao_dolar_df.withColumn("Mes", F.month("Data_Cotacao"))

media_mensal_df = cotacao_dolar_df.groupBy("Ano", "Mes").agg(
    F.round(F.avg("Cotacao_Compra"), 2).alias("Media_Mensal_Compra"),
    F.round(F.avg("Cotacao_Venda"), 2).alias("Media_Mensal_Venda")
).orderBy(F.col("Ano").desc(), F.col("Mes").desc())

display(media_mensal_df)

## 2.0 Carga no catálogo de metadados

### 2.1 Criação do Banco de Dados para Conectar Ferramenta de Visualização

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS Indicadores_Cotacao

### 2.2 Salvando tabela como Delta

In [0]:
cotacao_dolar_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("Indicadores_Cotacao.Media_Mensal")

### 2.3 Visualizando as tabelas do banco de dados

In [0]:
%sql

SHOW TABLES IN Indicadores_Cotacao

## 3.0 Carga no sistema de arquivos (diretório)

### 3.1 Salvando o DataFrame em formato Delta

In [0]:
cotacao_dolar_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save("dbfs:/mnt/Cotacao_Dolar/Gold/Media_Mensal")

### 3.2 Visualizando arquivo na Camada Gold

In [0]:
%fs
ls dbfs:/mnt/Cotacao_Dolar/Gold/Media_Mensal